第二部分：PyTorch(进阶)

In [9]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

Point1:模型定义方式
PyTorch中自定义模型主要通过一下三种方式
- Sequential
- ModuleList
- ModuleDict

In [10]:
#讲解点：使用ordered dict更有助于一目了然模型结构，对于之后的模型修改也非常有帮助
##Sequential:Direct list(直接列出来)两种定义方式
import torch.nn as nn
net1=nn.Sequential(#做一个两层神经网络
       nn.Linear(784,256),#输入为784
       nn.ReLU(),#损失函数
       nn.Linear(256,10),
       )
print(net1)#两个Linear的映射层，一个Relu激活层

Sequential(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [11]:
##Sequential:Ordered Dict#带顺序的字典
import collections
import torch.nn as nn
net2=nn.Sequential(collections.OrderedDict([
       ('fcl',nn.Linear(784,256)),
       ('relul',nn.ReLU()),
       ('fc2',nn.Linear(256,10))
       ]))
print(net2)
#两个一样的

Sequential(
  (fcl): Linear(in_features=784, out_features=256, bias=True)
  (relul): ReLU()
  (fc2): Linear(in_features=256, out_features=10, bias=True)
)


In [12]:
#试一下
a=torch.rand(4,784)#随机产生一个tensor
out1=net1(a)#输出结果
out2=net2(a)
print(out1.shape==out2.shape,out1.shape)

True torch.Size([4, 10])


In [22]:
#ModuleList
net3=nn.ModuleList([nn.Linear(784,256),nn.ReLU()])
net3.append(nn.Linear(256,10))#类似list的append操作
print(net3[-1])
print(net3)

Linear(in_features=256, out_features=10, bias=True)
ModuleList(
  (0): Linear(in_features=784, out_features=256, bias=True)
  (1): ReLU()
  (2): Linear(in_features=256, out_features=10, bias=True)
)


In [25]:
#注意ModuleList 并没有定义一个网络，他只是将不同的模块储存在一起，此处此处应报错
#out3=net3(a)

In [27]:
class Net3(nn.Module):
    def __init__(self):#先初始化
        super().__init__()#利用初始化定义模型层
        self.modulelist=nn.ModuleList([nn.Linear(784,256),nn.ReLU()])
        self.modulelist.append(nn.Linear(256,10))
    def forward(self,x):#得到前向传播的关系
        for layer in self.modulelist:
            x=layer(x)
        return x
net3_ = Net3()
out3_ = net3_(a)
print(out3_.shape)#得到结果

torch.Size([4, 10])


In [29]:
#ModuleDict
net4=nn.ModuleDict({#做一个两层神经网络
       'linear':nn.Linear(784,256),#输入为784
       'act':nn.ReLU(),#损失函数
})
net4['output']=nn.Linear(256,10)#添加第三层
print(net4['linear'])
print(net4.output)#两个Linear的映射层，一个Relu激活层

Linear(in_features=784, out_features=256, bias=True)
Linear(in_features=256, out_features=10, bias=True)


In [ ]:
#同样的，ModuleDict并没有定义一个网络，他只是将不同的模块储存在一起，此处应报错
#正确使用的方法同上
out4=net(a)

## 利用模块快速搭建复炸网路
      快速构建U-Net
      定义好几个模块，然后串起来，模块可以复用，U-Net每一个小块都有两次卷积的操作

组成U-Net的模型主要包括：
- 每个子块内部的两次卷积（Double Convolution)
- 左侧模型块之间的下采样链接，及最大池化（Max pooling）
- 右侧模型块之间的上采样连接（Up sampling）
- 输出层的处理
      除模型块外，还有模型块之间的，横向链接，输入和U-Net底部的链接计算，通过forward实现

In [30]:
import os
import numpy as np
import collections
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

In [33]:
class DoubleConv(nn.Module):#双次卷积
    '''(convolution =>[BN]>=ReLU) *2'''
    def __init__(self,in_channels,out_channels,mid_channels=None):#三个通道
        super().__init__()
        if not mid_channels:
            mid_channels=out_channels
        self.double_conv=nn.Sequential(
            nn.Conv2d(in_channels,mid_channels,kernel_size=3,padding=1,bias=False),#第一次卷积
            nn.BatchNorm1d(mid_channels),#忘了这是干啥来着
            nn.ReLU(inplace=True),#激活
            nn.Conv2d(mid_channels,out_channels,kernel_size=3,padding=1,bias=False),#第二次卷积
            nn.BatchNorm1d(out_channels),
            nn.ReLU(inplace=True)
        )
    def forward(self,x):
        return self.double_conv(x)

In [ ]:
class Down(nn.Module):
    '''Downscaling with maxpool then double conv'''
    def __init__(self,in_channel,out_channels):
        super().__init__()

In [ ]:
class Up(nn.Module):
    '''UPscaling then don=uble conv'''
    def __init__(self,in_channel,out_channels,bilinear=True):
        super.maxpool_conv=nn.Sequential(
        )